# Creating Estimators in tf.estimator

If you are getting started using SageMaker, check [iris-dnn-classifier-using-estimators/tutorial.ipynb](iris-dnn-classifier-using-estimators/tutorial.ipynb) first.

This tutorial covers how to create your own `Estimator` using the building
blocks provided in `tf.estimator`, which will predict the ages of
[abalones](https://en.wikipedia.org/wiki/Abalone) based on their physical
measurements. You'll learn how to do the following:

*   Instantiate an `Estimator`
*   Construct a custom model function
*   Configure a neural network using `tf.feature_column` and `tf.layers`
*   Choose an appropriate loss function from `tf.losses`
*   Define a training op for your model
*   Generate and return predictions

## An Abalone Age Predictor

It's possible to estimate the age of an
[abalone](https://en.wikipedia.org/wiki/Abalone) (sea snail) by the number of
rings on its shell. However, because this task requires cutting, staining, and
viewing the shell under a microscope, it's desirable to find other measurements
that can predict age.

The [Abalone Data Set](https://archive.ics.uci.edu/ml/datasets/Abalone) contains
the following
[feature data](https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.names)
for abalone:

| Feature        | Description                                               |
| -------------- | --------------------------------------------------------- |
| Length         | Length of abalone (in longest direction; in mm)           |
| Diameter       | Diameter of abalone (measurement perpendicular to length; in mm)|
| Height         | Height of abalone (with its meat inside shell; in mm)     |
| Whole Weight   | Weight of entire abalone (in grams)                       |
| Shucked Weight | Weight of abalone meat only (in grams)                    |
| Viscera Weight | Gut weight of abalone (in grams), after bleeding          |
| Shell Weight   | Weight of dried abalone shell (in grams)                  |

The label to predict is number of rings, as a proxy for abalone age.

## Let's start by setting up the environment.

In [1]:
import os
import sagemaker

sagemaker_session = sagemaker.Session()

# Replace with a role (either name or full arn) that gives SageMaker access to S3 and cloudwatch
role='SageMakerRole'

## Uploading the data

In [ ]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/abalone')

# Complete source code

In [ ]:
!cat 'abalone.py'

## Instantiating an Estimator

When defining a model using one of tf.estimator's provided classes, such as
`DNNClassifier`, you supply all the configuration parameters right in the
constructor, e.g.:

```python
my_nn = tf.estimator.DNNClassifier(feature_columns=[age, height, weight],
                                   hidden_units=[10, 10, 10],
                                   activation_fn=tf.nn.relu,
                                   dropout=0.2,
                                   n_classes=3,
                                   optimizer="Adam")
```

You don't need to write any further code to instruct TensorFlow how to train the
model, calculate loss, or return predictions; that logic is already baked into
the `DNNClassifier`.

By contrast, when you're creating your own estimator from scratch, the
constructor accepts just two high-level parameters for model configuration,
`model_fn` and `params`:

```python
nn = tf.estimator.Estimator(model_fn=model_fn, params=model_params)
```

*   `model_fn`: A function object that contains all the aforementioned logic to
    support training, evaluation, and prediction. You are responsible for
    implementing that functionality. The next section, [Constructing the
    `model_fn`](#constructing-modelfn) covers creating a model function in
    detail.

*   `params`: An optional dict of hyperparameters (e.g., learning rate, dropout)
    that will be passed into the `model_fn`.

Note: Just like `tf.estimator`'s predefined regressors and classifiers, the
`Estimator` initializer also accepts the general configuration arguments
`model_dir` and `config`.

For the abalone age predictor, the model will accept one hyperparameter:
learning rate. Define `LEARNING_RATE` as a constant at the beginning of your
code (highlighted in bold below), right after the logging configuration:

<pre class="prettyprint"><code class="lang-python">tf.logging.set_verbosity(tf.logging.INFO)

<strong># Learning rate for the model
LEARNING_RATE = 0.001</strong></code></pre>

Note: Here, `LEARNING_RATE` is set to `0.001`, but you can tune this value as
needed to achieve the best results during model training.

Then, add the following code to `main()`, which creates the dict `model_params`
containing the learning rate and instantiates the `Estimator`:

```python
# Set model params
model_params = {"learning_rate": LEARNING_RATE}

# Instantiate Estimator
nn = tf.estimator.Estimator(model_fn=model_fn, params=model_params)
```

## Constructing the `model_fn`

The basic skeleton for an `Estimator` API model function looks like this:

```python
def model_fn(features, labels, mode, params):
   # Logic to do the following:
   # 1. Configure the model via TensorFlow operations
   # 2. Define the loss function for training/evaluation
   # 3. Define the training operation/optimizer
   # 4. Generate predictions
   # 5. Return predictions/loss/train_op/eval_metric_ops in EstimatorSpec object
   return EstimatorSpec(mode, predictions, loss, train_op, eval_metric_ops)
```

The `model_fn` must accept three arguments:

*   `features`: A dict containing the features passed to the model via
    `input_fn`.
*   `labels`: A `Tensor` containing the labels passed to the model via
    `input_fn`. Will be empty for `predict()` calls, as these are the values the
    model will infer.
*   `mode`: One of the following tf.estimator.ModeKeys string values
    indicating the context in which the model_fn was invoked:
    *   `tf.estimator.ModeKeys.TRAIN` The `model_fn` was invoked in training
        mode, namely via a `train()` call.
    *   `tf.estimator.ModeKeys.EVAL`. The `model_fn` was invoked in
        evaluation mode, namely via an `evaluate()` call.
    *   `tf.estimator.ModeKeys.PREDICT`. The `model_fn` was invoked in
        predict mode, namely via a `predict()` call.

`model_fn` may also accept a `params` argument containing a dict of
hyperparameters used for training (as shown in the skeleton above).

The body of the function performs the following tasks (described in detail in the
sections that follow):

*   Configuring the model for the abalone predictor, this will be a neural
    network.
*   Defining the loss function used to calculate how closely the model's
    predictions match the target values.
*   Defining the training operation that specifies the `optimizer` algorithm to
    minimize the loss values calculated by the loss function.

The `model_fn` must return a tf.estimator.EstimatorSpec
object, which contains the following values:

*   `mode` (required). The mode in which the model was run. Typically, you will
    return the `mode` argument of the `model_fn` here.

*   `predictions` (required in `PREDICT` mode). A dict that maps key names of
    your choice to `Tensor`s containing the predictions from the model, e.g.:

    ```python
    predictions = {"results": tensor_of_predictions}
    ```

    In `PREDICT` mode, the dict that you return in `EstimatorSpec` will then be
    returned by `predict()`, so you can construct it in the format in which
    you'd like to consume it.


*   `loss` (required in `EVAL` and `TRAIN` mode). A `Tensor` containing a scalar
    loss value: the output of the model's loss function (discussed in more depth
    later in [Defining loss for the model](https://github.com/tensorflow/tensorflow/blob/eb84435170c694175e38bfa02751c3ef881c7a20/tensorflow/docs_src/extend/estimators.md#defining-loss)) calculated over all
    the input examples. This is used in `TRAIN` mode for error handling and
    logging, and is automatically included as a metric in `EVAL` mode.

*   `train_op` (required only in `TRAIN` mode). An Op that runs one step of
    training.

*   `eval_metric_ops` (optional). A dict of name/value pairs specifying the
    metrics that will be calculated when the model runs in `EVAL` mode. The name
    is a label of your choice for the metric, and the value is the result of
    your metric calculation. The tf.metrics
    module provides predefined functions for a variety of common metrics. The
    following `eval_metric_ops` contains an `"accuracy"` metric calculated using
    `tf.metrics.accuracy`:

    ```python
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels, predictions)
    }
    ```

    If you do not specify `eval_metric_ops`, only `loss` will be calculated
    during evaluation.

### Configuring a neural network with `tf.feature_column` and `tf.layers`

Constructing a [neural
network](https://en.wikipedia.org/wiki/Artificial_neural_network) entails
creating and connecting the input layer, the hidden layers, and the output
layer.

The input layer is a series of nodes (one for each feature in the model) that
will accept the feature data that is passed to the `model_fn` in the `features`
argument. If `features` contains an n-dimensional `Tensor` with all your feature
data, then it can serve as the input layer.
If `features` contains a dict of feature columns passed to
the model via an input function, you can convert it to an input-layer `Tensor`
with the tf.feature_column.input_layer function.

```python
input_layer = tf.feature_column.input_layer(
    features=features, feature_columns=[age, height, weight])
```

As shown above, `input_layer()` takes two required arguments:

*   `features`. A mapping from string keys to the `Tensors` containing the
    corresponding feature data. This is exactly what is passed to the `model_fn`
    in the `features` argument.
*   `feature_columns`. A list of all the `FeatureColumns`: `age`,
    `height`, and `weight` in the above example.

The input layer of the neural network then must be connected to one or more
hidden layers via an [activation
function](https://en.wikipedia.org/wiki/Activation_function) that performs a
nonlinear transformation on the data from the previous layer. The last hidden
layer is then connected to the output layer, the final layer in the model.
`tf.layers` provides the `tf.layers.dense` function for constructing fully
connected layers. The activation is controlled by the `activation` argument.
Some options to pass to the `activation` argument are:

*   `tf.nn.relu`. The following code creates a layer of `units` nodes fully
    connected to the previous layer `input_layer` with a
    [ReLU activation function](https://en.wikipedia.org/wiki/Rectifier_\(neural_networks\))
    (tf.nn.relu):

    ```python
    hidden_layer = tf.layers.dense(
        inputs=input_layer, units=10, activation=tf.nn.relu)
    ```

*   `tf.nn.relu6`. The following code creates a layer of `units` nodes fully
    connected to the previous layer `hidden_layer` with a ReLU activation
    function:

    ```python
    second_hidden_layer = tf.layers.dense(
        inputs=hidden_layer, units=20, activation=tf.nn.relu)
    ```

*   `None`. The following code creates a layer of `units` nodes fully connected
    to the previous layer `second_hidden_layer` with *no* activation function,
    just a linear transformation:

    ```python
    output_layer = tf.layers.dense(
        inputs=second_hidden_layer, units=3, activation=None)
    ```

Other activation functions are possible, e.g.:

```python
output_layer = tf.layers.dense(inputs=second_hidden_layer,
                               units=10,
                               activation_fn=tf.sigmoid)
```

The above code creates the neural network layer `output_layer`, which is fully
connected to `second_hidden_layer` with a sigmoid activation function
(tf.sigmoid).

Putting it all together, the following code constructs a full neural network for
the abalone predictor, and captures its predictions:

```python
def model_fn(features, labels, mode, params):
  """Model function for Estimator."""

  # Connect the first hidden layer to input layer
  # (features["x"]) with relu activation
  first_hidden_layer = tf.layers.dense(features["x"], 10, activation=tf.nn.relu)

  # Connect the second hidden layer to first hidden layer with relu
  second_hidden_layer = tf.layers.dense(
      first_hidden_layer, 10, activation=tf.nn.relu)

  # Connect the output layer to second hidden layer (no activation fn)
  output_layer = tf.layers.dense(second_hidden_layer, 1)

  # Reshape output layer to 1-dim Tensor to return predictions
  predictions = tf.reshape(output_layer, [-1])
  predictions_dict = {"ages": predictions}
  ...
```

Here, because you'll be passing the abalone `Datasets` using `numpy_input_fn`
as shown below, `features` is a dict `{"x": data_tensor}`, so
`features["x"]` is the input layer. The network contains two hidden
layers, each with 10 nodes and a ReLU activation function. The output layer
contains no activation function, and is
tf.reshape to a one-dimensional
tensor to capture the model's predictions, which are stored in
`predictions_dict`.

### Defining loss for the model

The `EstimatorSpec` returned by the `model_fn` must contain `loss`: a `Tensor`
representing the loss value, which quantifies how well the model's predictions
reflect the label values during training and evaluation runs. The tf.losses
module provides convenience functions for calculating loss using a variety of
metrics, including:

*   `absolute_difference(labels, predictions)`. Calculates loss using the
    [absolute-difference
    formula](https://en.wikipedia.org/wiki/Deviation_\(statistics\)#Unsigned_or_absolute_deviation)
    (also known as L<sub>1</sub> loss).

*   `log_loss(labels, predictions)`. Calculates loss using the [logistic loss
    forumula](https://en.wikipedia.org/wiki/Loss_functions_for_classification#Logistic_loss)
    (typically used in logistic regression).

*   `mean_squared_error(labels, predictions)`. Calculates loss using the [mean
    squared error](https://en.wikipedia.org/wiki/Mean_squared_error) (MSE; also
    known as L<sub>2</sub> loss).

The following example adds a definition for `loss` to the abalone `model_fn`
using `mean_squared_error()` (in bold):

```python
def model_fn(features, labels, mode, params):
  """Model function for Estimator."""

  # Connect the first hidden layer to input layer
  # (features["x"]) with relu activation
  first_hidden_layer = tf.layers.dense(features["x"], 10, activation=tf.nn.relu)

  # Connect the second hidden layer to first hidden layer with relu
  second_hidden_layer = tf.layers.dense(
      first_hidden_layer, 10, activation=tf.nn.relu)

  # Connect the output layer to second hidden layer (no activation fn)
  output_layer = tf.layers.dense(second_hidden_layer, 1)

  # Reshape output layer to 1-dim Tensor to return predictions
  predictions = tf.reshape(output_layer, [-1])
  predictions_dict = {"ages": predictions}


  # Calculate loss using mean squared error
  loss = tf.losses.mean_squared_error(labels, predictions)
  ...
```

See the [tf.losses$API guide](https://www.tensorflow.org/api_docs/python/tf/losses) for a
full list of loss functions and more details on supported arguments and usage.

Supplementary metrics for evaluation can be added to an `eval_metric_ops` dict.
The following code defines an `rmse` metric, which calculates the root mean
squared error for the model predictions. Note that the `labels` tensor is cast
to a `float64` type to match the data type of the `predictions` tensor, which
will contain real values:

```python
eval_metric_ops = {
    "rmse": tf.metrics.root_mean_squared_error(
        tf.cast(labels, tf.float64), predictions)
}
```

### Defining the training op for the model

The training op defines the optimization algorithm TensorFlow will use when
fitting the model to the training data. Typically when training, the goal is to
minimize loss. A simple way to create the training op is to instantiate a
`tf.train.Optimizer` subclass and call the `minimize` method.

The following code defines a training op for the abalone `model_fn` using the
loss value calculated in [Defining Loss for the Model](https://github.com/tensorflow/tensorflow/blob/eb84435170c694175e38bfa02751c3ef881c7a20/tensorflow/docs_src/extend/estimators.md#defining-loss), the
learning rate passed to the function in `params`, and the gradient descent
optimizer. For `global_step`, the convenience function
tf.train.get_global_step takes care of generating an integer variable:

```python
optimizer = tf.train.GradientDescentOptimizer(
    learning_rate=params["learning_rate"])
train_op = optimizer.minimize(
    loss=loss, global_step=tf.train.get_global_step())
```

For a full list of optimizers, and other details, see the
@{$python/train#optimizers$API guide}.

### The complete abalone `model_fn`

Here's the final, complete `model_fn` for the abalone age predictor. The
following code configures the neural network; defines loss and the training op;
and returns a `EstimatorSpec` object containing `mode`, `predictions_dict`, `loss`,
and `train_op`:

```python
def model_fn(features, labels, mode, params):
  """Model function for Estimator."""

  # Connect the first hidden layer to input layer
  # (features["x"]) with relu activation
  first_hidden_layer = tf.layers.dense(features["x"], 10, activation=tf.nn.relu)

  # Connect the second hidden layer to first hidden layer with relu
  second_hidden_layer = tf.layers.dense(
      first_hidden_layer, 10, activation=tf.nn.relu)

  # Connect the output layer to second hidden layer (no activation fn)
  output_layer = tf.layers.dense(second_hidden_layer, 1)

  # Reshape output layer to 1-dim Tensor to return predictions
  predictions = tf.reshape(output_layer, [-1])

  # Provide an estimator spec for `ModeKeys.PREDICT`.
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions={"ages": predictions})

  # Calculate loss using mean squared error
  loss = tf.losses.mean_squared_error(labels, predictions)

  # Calculate root mean squared error as additional eval metric
  eval_metric_ops = {
      "rmse": tf.metrics.root_mean_squared_error(
          tf.cast(labels, tf.float64), predictions)
  }

  optimizer = tf.train.GradientDescentOptimizer(
      learning_rate=params["learning_rate"])
  train_op = optimizer.minimize(
      loss=loss, global_step=tf.train.get_global_step())

  # Provide an estimator spec for `ModeKeys.EVAL` and `ModeKeys.TRAIN` modes.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      loss=loss,
      train_op=train_op,
      eval_metric_ops=eval_metric_ops)
```

# Submitting script for training

In [ ]:
from sagemaker.tensorflow import TensorFlow

abalone_estimator = TensorFlow(entry_point='abalone.py',
                                  role=role,
                                  hyperparameters={'training_steps': 100, 'learning_rate': 0.001},
                                  train_instance_count=1,
                                  train_instance_type='ml.c4.xlarge')

abalone_estimator.fit(inputs)

`estimator.fit` will deploy a script in a container for training and returs the SageMaker model name using the following arguments:

*   `framework=tensorflow`. Tells submit_training that it is a tensorflow container.
*   `script="abalone.py"`. The relative path to the script that will be deployed to the container.
*   `data="s3://bucket_name/abalone-estimator-training"`. The S3 location of the bucket that we uploaded earlier.
*   `role="maeve-pullrole"`. AWS role that gives your account access to SageMaker training and hosting
* `hyperparameters={'training_steps' : 100}`. Model and training hyperparameters. 

Running the code block above will do the following actions:
* deploy your script in a container with tensorflow installed
* copy the data from the bucket to the container
* instantiate the tf.estimator
* train the estimator with 10 training steps
* save the estimator model

# Submiting a trained model for hosting


In [ ]:
abalone_predictor = abalone_estimator.deploy(initial_instance_count=1,
                                             instance_type='ml.c4.xlarge')

`abalone_estimator.deploy` deploys the trained model in a container ready for production.

# Invoking the endpoint

In [ ]:
import tensorflow as tf
import numpy as np

prediction_set = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=os.path.join('data/abalone_predict.csv'), target_dtype=np.int, features_dtype=np.float32)

data = prediction_set.data[0]
tensor_proto = tf.make_tensor_proto(values=np.asarray(data), shape=[1, len(data)], dtype=tf.float32)

In [ ]:
abalone_predictor.predict(tensor_proto)

# Deleting the endpoint

In [ ]:
abalone_estimator.delete_endpoint()